In [1]:
import numpy as np
import pandas as pd

# Dataset:

Netflix Price Dataset: https://www.kaggle.com/datasets/netflix-inc/netflix-prize-data

# Load Dataset

using with open() to open csv file as:
1. Deal with letters as À, É, ..., etc.
2. Some movies habe multiple titles seperarted by commas.

In [9]:
movies_df = pd.DataFrame(columns=['id', 'year', 'title'])

with open('archive/movie_titles.csv', 'rb') as file:
    for l in file.readlines():
        line_str = l.decode('utf-8', errors="ignore")
        columns = line_str.split(',')
        id_val, date_val, title_val = columns[:3]
        title_val += ', '.join(columns[3:]).strip()
        movies_df = pd.concat([movies_df, pd.DataFrame([{'id':id_val,'year': date_val,'title':title_val}])], ignore_index=True)
        
movies_df.head(3)

,id,year,title
0,1,2003,Dinosaur Planet\n
1,2,2004,Isle of Man TT 2004 Review\n
2,3,1997,Character\n
